# Run simple CMB fits, using the CMB parameterization

In [1]:
import numpy as np
import sys
from pathlib import Path

In [2]:
# figure out configuration for laptop vs NERSC
laptop=True
if laptop:
    cosmo_dir = '/Users/afont/Desktop/bao-cosmology'
else:
    cosmo_dir = '/global/cfs/cdirs/desi/users/font/bao-cosmology'
sys.path.insert(1, f'{cosmo_dir}/py/')
import write_cobaya_configs as write
import compressed_cmb as cmb

In [3]:
# set configuration options for these runs
cosmo_path = Path(cosmo_dir)
# store chains here
runs_path = cosmo_path / 'runs'

In [8]:
# run analysis with CMB only
def run_cmb(model, thetastar=True, ombh2=True, ombch2=True, debug=True, submit=True):
    label = 'cmb'
    if thetastar:
        label += '_thetastar'
    if ombh2:
        label += '_ombh2'
    if ombch2:
        label += '_ombch2'
    print('will use label',label)
    cmb_like = cmb.get_compressed_cmb_likelihood(thetastar=thetastar, ombh2=ombh2, ombch2=ombch2)
    # do not include the last BAO data, since that is a fake CMB theta_star (for now)
    config_file = write.write_cmb_cobaya_config(model=model, run_label=label, 
                                                cmb_like=cmb_like, runs_path=runs_path)
    print(config_file)
    if laptop:
        updated_info, sampler = write.run_cobaya(config_file, force=True, resume=False)
        return updated_info, sampler
    else:
        command = write.send_cobaya_job(config_file, debug=debug, submit=submit)
        print(command)
        return command 

In [ ]:
results = run_cmb(model='w0wa', thetastar=True, ombh2=True, ombch2=True, debug=True, submit=True)